##### Configuración

In [1]:
## Celda de configuración

# Librerias
import random
import time
import sys
import os

# Instancias y algoritmos
sys.path.append(os.path.abspath("../Instances"))
from InsKnapsack import generar_datos, visualizar, report

# Programación Dinámica
sys.path.append(os.path.abspath("../DP/Env"))         # entorno del problema
sys.path.append(os.path.abspath("../DP/Algorithms/")) # algoritmos RL / DP
sys.path.append(os.path.abspath("../DP/Visual/"))     # visualización de políticas / valores

from Knapsack import KnapsackEnv                      # Clase que define el entorno tipo mochila

from value_iteration import value_iteration           # iteración de valores
from policy_evaluation import policy_evaluation       # evaluación de políticas
from policy_iteration import policy_iteration         # iteración de políticas

from value_states import value_states_visual          # visualización de V(s)
from policy_dag import draw_policy_dag                # grafo de política óptima

# MinObras

El Ministerio de Transporte (MinTransporte) cuenta con un presupuesto de **100 millones de pesos** para ejecutar obras de infraestructura vial, con el fin de generar desarrollo en Bogotá. D.C.

MinTransporte le va a proporcionar datos que contiene información sobre:

- **Costo de ejecución** de cada proyecto (en millones de pesos)
- **Número de empleos generados** (en miles)
- **Ubicación geográfica** del proyecto (latitud y longitud)

El objetivo del Ministerio es **maximizar la cantidad total de empleos generados**, respetando el límite presupuestal.

In [2]:
# 1. TODO Defina el número de obras a usar.
NUM_OBRAS = 25

# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Generar Obras.
obras = generar_datos(NUM_OBRAS)
visualizar(obras)

# . Guardar resultados.
resultados = []

## Optimización (MIP)

### Formulación

#### 1. Conjuntos  
$O$: conjunto de obras disponibles.

#### 2. Parámetros  
$c_i$: costo de ejecución (en millones de pesos) de la obra $i \in O$  
$e_i$: empleos generados (en miles) por la realización de la obra $i \in O$  
$p$: presupuesto total disponible (en millones de pesos)

#### 3. Variables de decisión  
$$
x_i =
\begin{cases}
1, & \text{si la obra } i \in O \text{ se lleva a cabo} \\
0, & \text{en caso contrario}
\end{cases}
$$

#### 4. Función objetivo  
Maximizar el total de empleos generados:
$$
\max \sum_{i \in O} e_i \cdot x_i
$$

#### 5. Restricciones  

**(1) Presupuesto disponible:**  
$$
\sum_{i \in O} c_i \cdot x_i \leq p
$$

**(2) Naturaleza de las variables:**  
$$
x_i \in \{0, 1\}, \quad \forall i \in O
$$

Donde:

- La restricción (1) garantiza que no se exceda el presupuesto disponible.  
- La restricción (2) establece la naturaleza binaria de las variables de decisión.


In [3]:
# 2. TODO Extraiga los parámetros relevantes desde el DataFrame 'obras' para
# construir los insumos del modelo:
#   • Conjunto de decisiones
#   • Costos (c)
#   • Beneficios (empleos generados, e)
#   • Presupuesto total disponible

# Conjuntos
O = list(obras.index)

# Parámetros
c = dict(obras['Costo de ejecución (en millones de pesos)'])
e = dict(obras['# de empleos generados (en miles)'])

# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Presupuesto
p = min(len(O), 100)

In [4]:
# 3. TODO Complete la siguiente función

import pulp as lp

def optimizacion_knapsack(O, c, e, p):

    # Definir el problema.
    model = lp.LpProblem("Obras", lp.LpMaximize)

    # Variables de decisión
    x = {i: lp.LpVariable(f'x_{i}', lowBound=0, cat=lp.LpBinary) for i in O}

    # Función Objetivo
    model += lp.lpSum(e[i] * x[i] for i in O)

    # Restricciones
    # ▸ Presupuesto total
    model += lp.lpSum(c[i] * x[i] for i in O) <= p

    # ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

    # Resolver
    solver = lp.getSolver('PULP_CBC_CMD', msg=False)
    model.solve(solver)

    # Reportar
    print(f'El optimizador llegó a una solución: {lp.LpStatus[model.status]}.')
    return model, x

In [ ]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr el optimizador.
start = time.perf_counter()
model, x = optimizacion_knapsack(O, c, e, p)
end = time.perf_counter()

# . Extarer resultados
if lp.LpStatus[model.status] == 'Optimal':

    obj_lp = round(lp.value(model.objective), 3)                        #   ▸ Función Objetivo
    obras_selec_lp = [i for i in O if x[i].varValue == 1]               #   ▸ Obras seleccionadas
    presupuesto_usado_lp = round(sum(c[i] for i in obras_selec_lp), 2)  #   ▸ Presupuesto total utilizado
    
    # . Imprimir resultados
    print("Objetos seleccionados:")
    for idx in obras_selec_lp:
        w, v = c[idx], e[idx]
        print(f"  • Obj {idx:>2}: peso={w}, valor={v}")

    print(f"FO (valor total):    {obj_lp}")
    print(f"Presupuesto usado:   {presupuesto_usado_lp}/{p}")

    report(obras, obras_selec_lp, obj_lp, presupuesto_usado_lp, p)      #   ▸ Reportar

    # . Almacenar resultados
    resultados.append({
        "Método"             : "LP",
        "Empleos generados"  : obj_lp,
        "Obras seleccionadas": len(obras_selec_lp),
        "Presupuesto usado"  : presupuesto_usado_lp,
        "Tiempo (s)"         : end - start
    })

else:
    print(f"ERROR: Algo quedo mal con el modelo de optimización: {lp.LpStatus[model.status]}")

El optimizador llegó a una solución: Optimal.
Objetos seleccionados:
  • Obj  8: peso=4.0, valor=11.0
  • Obj 10: peso=1.0, valor=11.0
  • Obj 12: peso=8.0, valor=15.0
  • Obj 13: peso=3.0, valor=7.0
  • Obj 21: peso=2.0, valor=12.0
  • Obj 22: peso=6.0, valor=11.0
  • Obj 23: peso=1.0, valor=12.0
FO (valor total):    79.0
Presupuesto usado:   25.0/25


## Programación Dinámica

In [6]:
# 4. TODO Mapee cada uno de los siguientes datos usados por el ambiente.

weights  = list(c.values())  # pesos de los ítems: costos (millones de COP)
values   = list(e.values())  # beneficios de los ítems: empleos (miles)
capacity = p                 # capacidad total: presupuesto disponible

# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Crear el ambiente
env = KnapsackEnv(weights, values, capacity)
print(env)

KnapsackEnv(#_Objetos = 25, Capacidad = 25, #_Estados = 676)


### Policy Evaluation

```text
# Calcula Vπ para una política fija π
# Entradas:
#   S          : conjunto de estados
#   π(s)       : política dada (acción para cada s)
#   p(s',r|s,a): modelo de transición y recompensa
#   γ          : factor de descuento (0 ≤ γ ≤ 1)
#   θ          : umbral de convergencia
# Salida:
#   Vπ(s)      : valor esperado para cada estado bajo π
# -----------------------------------------------------

# Inicialización
V(s) ← 0  para todo s ∈ S

# Evaluación iterativa
repetir
    Δ ← 0
    para cada estado s ∈ S:
        v ← V(s)                                    # valor anterior
        a ← π(s)                                    # acción dictada por la política
        V(s) ← Σ_{s',r} p(s',r | s,a) · [ r + γ · V(s') ]
        Δ ← max(Δ, |v − V(s)|)
hasta que Δ < θ

return V
```

In [7]:
# 5. TODO DEFINIR POLÍTICA: Escriba la siguiente política:
# Para cada estado s = (i, capacidad restante) la acción: 'take' si está permitida; si no, 'skip'.
#       * De ser necesario, explore el ambiente.

propose_policy = {s: ('take' if 'take' in env.actions(s) else 'skip') for s in env.state_space()}

# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

policy = propose_policy.copy()

In [8]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr
t0 = time.perf_counter()
V = policy_evaluation(env, propose_policy)
elapsed = time.perf_counter() - t0

# . Reportar
valor_total, objetos_tomados, peso_total = env.report_from_policy(propose_policy)
report(obras, objetos_tomados, valor_total, peso_total, p)

# . Almacenar resultados
resultados.append({
    "Método"             : "Policy evaluation",
    "Empleos generados"  : round(valor_total, 3),
    "Obras seleccionadas": len(objetos_tomados),
    "Presupuesto usado"  : round(peso_total, 2),
    "Tiempo (s)"         : elapsed
})

Objetos seleccionados:
  • Obj  0: peso=8.0, valor=13.0
  • Obj  1: peso=7.0, valor=12.0
  • Obj  2: peso=2.0, valor=4.0
  • Obj  3: peso=7.0, valor=12.0
  • Obj 10: peso=1.0, valor=11.0
FO (valor total):    52.0
Presupuesto usado:   25.0/25


### Policy Iteration

```text
# Alterna evaluación y mejora hasta que la política se estabiliza
# Entradas:
#   S, A(s), p(s',r|s,a), γ, θ
# Salidas:
#   π*  : política óptima
#   V*  : función de valor óptima
# -------------------------------------------

# 1. Inicialización
π(s) ← acción cualquiera  para todo s ∈ S
V(s) ← 0

bucle:
    # 2. Evaluar la política actual π  (igual que el bloque anterior)
    repetir
        Δ ← 0
        para cada s ∈ S:
            v ← V(s)
            a ← π(s)
            V(s) ← Σ_{s',r} p(s',r | s,a) · [ r + γ · V(s') ]
            Δ ← max(Δ, |v − V(s)|)
    hasta que Δ < θ

    # 3. Mejorar la política
    policy_stable ← true
    para cada s ∈ S:
        old ← π(s)
        π(s) ← argmax_a Σ_{s',r} p(s',r | s,a) · [ r + γ · V(s') ]
        si old ≠ π(s):
            policy_stable ← false

    si policy_stable:
        break

return π, V

```

In [9]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr
t0 = time.perf_counter()
star_policy, V_star = policy_iteration(env, policy)
elapsed = time.perf_counter() - t0

# . Reportar
valor_total, objetos_tomados, peso_total = env.report_from_policy(star_policy)
report(obras, objetos_tomados, valor_total, peso_total, p)

# . Almacenar resultados
resultados.append({
    "Método"             : "Policy iteration",
    "Empleos generados"  : round(valor_total, 3),
    "Obras seleccionadas": len(objetos_tomados),
    "Presupuesto usado"  : round(peso_total, 2),
    "Tiempo (s)"         : elapsed
})

Objetos seleccionados:
  • Obj  8: peso=4.0, valor=11.0
  • Obj 10: peso=1.0, valor=11.0
  • Obj 12: peso=8.0, valor=15.0
  • Obj 13: peso=3.0, valor=7.0
  • Obj 21: peso=2.0, valor=12.0
  • Obj 22: peso=6.0, valor=11.0
  • Obj 23: peso=1.0, valor=12.0
FO (valor total):    79.0
Presupuesto usado:   25.0/25


### Value Iteration

```text
# Aproxima V* directamente y luego deriva π*
# Entradas:
#   S, A(s), p(s',r|s,a), γ, θ
# Salidas:
#   π* : política óptima
#   V* : función de valor óptima
# -------------------------------------------

# Fase 1: aproximar V*
V(s) ← 0  para todo s ∈ S

repetir
    Δ ← 0
    para cada s ∈ S:
        v ← V(s)
        V(s) ← max_a Σ_{s',r} p(s',r | s,a) · [ r + γ · V(s') ]
        Δ ← max(Δ, |v − V(s)|)
hasta que Δ < θ

# Fase 2: derivar π* usando V*
para cada s ∈ S:
    π*(s) ← argmax_a Σ_{s',r} p(s',r | s,a) · [ r + γ · V(s') ]

return π*, V
```

In [10]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr
t0 = time.perf_counter()
opt_policy, V_opt = value_iteration(env)
elapsed = time.perf_counter() - t0

# . Reportar
valor_total, objetos_tomados, peso_total = env.report_from_policy(opt_policy)
report(obras, objetos_tomados, valor_total, peso_total, p)

# . Almacenar resultados
resultados.append({
    "Método"             : "Value iteration",
    "Empleos generados"  : round(valor_total, 3),
    "Obras seleccionadas": len(objetos_tomados),
    "Presupuesto usado"  : round(peso_total, 2),
    "Tiempo (s)"         : elapsed
})

Objetos seleccionados:
  • Obj  8: peso=4.0, valor=11.0
  • Obj 10: peso=1.0, valor=11.0
  • Obj 12: peso=8.0, valor=15.0
  • Obj 13: peso=3.0, valor=7.0
  • Obj 21: peso=2.0, valor=12.0
  • Obj 22: peso=6.0, valor=11.0
  • Obj 23: peso=1.0, valor=12.0
FO (valor total):    79.0
Presupuesto usado:   25.0/25


## Heurísticas

#### Greedy

```text
# ENTRADAS -------------------------------------------------
#   C               : conjunto de todos los candidatos.
#   es_factible(s,c): True si añadir c a la solución parcial s sin violar las #                       restricciones del problema.
#   criterio(c)     : puntuación de prioridad (cuanto mayor, mejor).
#   objetivo(s)     : función que evalúa la solución final s.
#
# SALIDAS  -----------------------------------------------
#   sol             : subconjunto final de candidatos elegidos.
#   valor_objetivo  : objetivo(sol).
# --------------------------------------------------------

# 1. Inicializar solución vacía
sol ← ∅

# 2. Ordenar candidatos por prioridad (descendente)
C_ordenado ← ordenar( C ,   por = criterio ,   modo = 'desc' )

# 3. Recorrer la lista ordenada e ir añadiendo si es factible
para cada candidato c en C_ordenado:
    si es_factible( sol , c ):
        sol ← sol ∪ { c }

# 4. Calcular el valor de la solución
valor_objetivo ← objetivo( sol )

# 5. Devolver solución y su valor
return sol , valor_objetivo
```


In [ ]:
# 6. TODO Complete la función.

def greedy_knapsack(O, c, e, p, criterio):

    # . Inicializar
    presupuesto_actual = 0.0
    obras_selec_greedy = []

    # . Ordenar según criterio
    obras_ordenadas = sorted(O, key=lambda i: criterio[i], reverse=True)

    # . Añadir si es factible
    for i in obras_ordenadas:
        if presupuesto_actual + c[i] <= p:
            obras_selec_greedy.append(i)
            presupuesto_actual += c[i]

    # . Resultados
    obj_greedy = round(sum(e[i] for i in obras_selec_greedy), 2)
    presupuesto_usado_greedy = round(presupuesto_actual, 2)

    # ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

    # . Devolver
    return obj_greedy, obras_selec_greedy, presupuesto_usado_greedy


In [12]:
# 7. TODO Defina el criterio deseado (como diccionario).
criterio = e

# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr
start = time.perf_counter()
obj_greedy, obras_selec_greedy, presupuesto_usado_greedy = greedy_knapsack(O, c, e, p, criterio)
end = time.perf_counter()

# . Reportar
print("Objetos seleccionados:")
for idx in obras_selec_greedy:
    w, v = c[idx], e[idx]
    print(f"  • Obj {idx:>2}: peso={w}, valor={v}")

print(f"FO (valor total):    {obj_greedy}")
print(f"Presupuesto usado:   {presupuesto_usado_greedy}/{p}")

report(obras, obras_selec_greedy, obj_greedy, presupuesto_usado_greedy, p)      #   ▸ Reportar


# . Almacenar resultados
resultados.append({
    "Método"             : "Greedy (e)",
    "Empleos generados"  : obj_greedy,
    "Obras seleccionadas": len(obras_selec_greedy),
    "Presupuesto usado"  : presupuesto_usado_greedy,
    "Tiempo (s)"         : end - start
})


Objetos seleccionados:
  • Obj 12: peso=8.0, valor=15.0
  • Obj  0: peso=8.0, valor=13.0
  • Obj 17: peso=8.0, valor=13.0
  • Obj 23: peso=1.0, valor=12.0
FO (valor total):    53.0
Presupuesto usado:   25.0/25


In [13]:
# 8. TODO Defina un nuevo criterio (como diccionario)
criterio = {i: e[i] / c[i] for i in O}

# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr
start = time.perf_counter()
obj_greedy, obras_selec_greedy, presupuesto_usado_greedy = greedy_knapsack(O, c, e, p, criterio)
end = time.perf_counter()

# . Reportar
print("Objetos seleccionados:")
for idx in obras_selec_greedy:
    w, v = c[idx], e[idx]
    print(f"  • Obj {idx:>2}: peso={w}, valor={v}")

print(f"FO (valor total):    {obj_greedy}")
print(f"Presupuesto usado:   {presupuesto_usado_greedy}/{p}")

report(obras, obras_selec_greedy, obj_greedy, presupuesto_usado_greedy, p)      #   ▸ Reportar


# . Almacenar resultados
resultados.append({
    "Método"             : "Greedy (e/c)",
    "Empleos generados"  : obj_greedy,
    "Obras seleccionadas": len(obras_selec_greedy),
    "Presupuesto usado"  : presupuesto_usado_greedy,
    "Tiempo (s)"         : end - start
})


Objetos seleccionados:
  • Obj 23: peso=1.0, valor=12.0
  • Obj 10: peso=1.0, valor=11.0
  • Obj 21: peso=2.0, valor=12.0
  • Obj  8: peso=4.0, valor=11.0
  • Obj 13: peso=3.0, valor=7.0
  • Obj  2: peso=2.0, valor=4.0
  • Obj 12: peso=8.0, valor=15.0
FO (valor total):    72.0
Presupuesto usado:   21.0/25


### Random

```text
# ENTRADAS -------------------------------------------------
#   C               : conjunto de todos los candidatos
#   es_factible(s,c): True si añadir c a la solución parcial s
#                     mantiene todas las restricciones
#   objetivo(s)     : función que puntúa la solución completa s
#
# SALIDAS  -----------------------------------------------
#   sol             : subconjunto final de candidatos elegidos
#   valor_objetivo  : objetivo(sol)
# --------------------------------------------------------

# 1. Inicializar solución vacía
sol ← ∅

# 2. Crear un orden aleatorio de candidatos
C_rand ← permutación_aleatoria( C )   # sin reemplazo

# 3. Recorrer la lista aleatoria e ir añadiendo si es factible
para cada candidato c en C_rand:
    si es_factible( sol , c ):
        sol ← sol ∪ { c }

# 4. Calcular el valor de la solución final
valor_objetivo ← objetivo( sol )

# 5. Devolver solución y su valor
return sol , valor_objetivo
```


In [ ]:
# 9. TODO Complete la función.
def random_knapsack(O, c, e, p):

    # . Inicialización
    presupuesto_actual = 0.0
    obras_selec_rand = []

    # . Aleatorizar
    obras_random = random.sample(O, len(O))

    # . Añadir si es factible
    for i in obras_random:
        if presupuesto_actual + c[i] <= p:
            obras_selec_rand.append(i)
            presupuesto_actual += c[i]

    # . Resultados
    obj_rand = round(sum(e[i] for i in obras_selec_rand), 2)
    presupuesto_usado_rand = round(presupuesto_actual, 2)

    # . Devolver
    return obj_rand, obras_selec_rand, presupuesto_usado_rand


In [15]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr
start = time.perf_counter()
obj_rand, obras_selec_rand, presupuesto_usado_rand = random_knapsack(O, c, e, p)
end = time.perf_counter()

# . Reportar
print("Objetos seleccionados:")
for idx in obras_selec_rand:
    w, v = c[idx], e[idx]
    print(f"  • Obj {idx:>2}: peso={w}, valor={v}")

print(f"FO (valor total):    {obj_rand}")
print(f"Presupuesto usado:   {presupuesto_usado_rand}/{p}")

report(obras, obras_selec_rand, obj_rand, presupuesto_usado_rand, p)      #   ▸ Reportar

# . Almacenar resultados
resultados.append({
    "Método"             : f"Random",
    "Empleos generados"  : obj_rand,
    "Obras seleccionadas": len(obras_selec_rand),
    "Presupuesto usado"  : presupuesto_usado_rand,
    "Tiempo (s)"         : end - start
})


Objetos seleccionados:
  • Obj  0: peso=8.0, valor=13.0
  • Obj 13: peso=3.0, valor=7.0
  • Obj  7: peso=7.0, valor=10.0
  • Obj 22: peso=6.0, valor=11.0
  • Obj 23: peso=1.0, valor=12.0
FO (valor total):    53.0
Presupuesto usado:   25.0/25


#### Swaps

```text
# ENTRADAS -------------------------------------------------
#   C           : conjunto total de candidatos
#   costo(i)    : costo asociado al candidato i
#   valor(i)    : beneficio (empleos, utilidad, etc.) del candidato i
#   C_max       : presupuesto máximo permitido
#   sol_inicial : conjunto factible de partida
#   n_intentos  : cuántos swaps probar antes de detenerse
#
# SALIDAS  -----------------------------------------------
#   mejor_sol   : mejor conjunto encontrado
#   mejor_valor : suma de valor(i) en mejor_sol
#   costo_final : suma de costo(i) en mejor_sol
# --------------------------------------------------------

# 1. Inicializar con la solución de partida
mejor_sol   ← sol_inicial
mejor_valor ← Σ_{i∈mejor_sol}  valor(i)

# 2. Repetir n_intentos veces
para t = 1 … n_intentos:

    # 2a. Elegir aleatoriamente un elemento dentro y otro fuera
    i ← elemento_aleatorio(  mejor_sol                 )
    j ← elemento_aleatorio(  C  \  mejor_sol           )

    # 2b. Construir la solución de prueba: quitar i y añadir j
    nueva_sol   ← ( mejor_sol  \  { i } )  ∪  { j }
    nuevo_costo ← Σ_{k∈nueva_sol}  costo(k)

    # 2c. Verificar factibilidad de presupuesto
    si nuevo_costo ≤ C_max entonces:

        nuevo_valor ← Σ_{k∈nueva_sol}  valor(k)

        # 2d. Aceptar solo si mejora el valor
        si nuevo_valor > mejor_valor entonces:
            mejor_sol   ← nueva_sol
            mejor_valor ← nuevo_valor
            # (opcional) reiniciar contador de intentos si se desea
            # n_intentos ← n_intentos   # mantener, si no se reinicia

# 3. Calcular costo final para reportar
costo_final ← Σ_{i∈mejor_sol}  costo(i)

# 4. Devolver mejor solución encontrada
return mejor_sol , mejor_valor , costo_final
```


In [ ]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

def swaps_knapsack(O, c, e, p, sol_inicial, n=100, report=False):

    # . Inicializar
    mejor_sol = sol_inicial.copy()
    mejor_obj = sum(e[i] for i in mejor_sol)

    # . 'n' Swaps
    for inter in range(1, n + 1):

        # ▸ Seleccionar una obra para quitar
        i = random.choice(mejor_sol)

        # ▸ Seleccionar una obra que no esté en la solución
        candidatos = list(set(O) - set(mejor_sol))
        j = random.choice(candidatos)

        # ▸ Proponer nueva solución: swap i → j
        nueva_sol = mejor_sol.copy()
        nueva_sol.remove(i)
        nueva_sol.append(j)

        # ▸ Calcular el costo total de la nueva solución
        costo = sum(c[k] for k in nueva_sol)

        # ▸ Verificar si respeta el presupuesto
        if costo <= p:

            # ▸ Calcular empleos generados por la nueva solución
            obj = sum(e[k] for k in nueva_sol)

            # ▸ Aceptar mejora si aumenta empleos
            if obj > mejor_obj:
                if report:
                    print(f'[Intento {inter}] Mejora aleatoria:')
                    print(f'  - Swap: {i} ➜ {j} (e: {e[i]} ➜ {e[j]}, c: {c[i]} ➜ {c[j]})')
                    print(f'  - Objetivo nuevo: {round(obj, 2)}')
                    print(f'  - Presupuesto: {round(costo, 2)}/{p}\n')

                mejor_sol, mejor_obj = nueva_sol, obj

    # Resultados
    presupuesto_usado = sum(c[i] for i in mejor_sol)

    # Devolver
    return mejor_obj, mejor_sol, presupuesto_usado

In [ ]:
# 10. TODO Defina cuantos intercambios desea hacer.
NUM_SWAPS = 10_000

# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Definir solución inicial
obj_rand, obras_selec_rand, presupuesto_usado_rand = random_knapsack(O, c, e, p)
sol_inicial = obras_selec_rand

# . Correr
start = time.perf_counter()
obj_rand_swaps, obras_selec_rand_swaps, presupuesto_usado_rand_swaps = swaps_knapsack(O, c, e, p, sol_inicial, n = NUM_SWAPS, report=False)
end = time.perf_counter()

# . Reportar
print("Objetos seleccionados:")
for idx in obras_selec_rand_swaps:
    w, v = c[idx], e[idx]
    print(f"  • Obj {idx:>2}: peso={w}, valor={v}")

print(f"FO (valor total):    {obj_rand_swaps}")
print(f"Presupuesto usado:   {presupuesto_usado_rand_swaps}/{p}")

report(obras, obras_selec_rand_swaps, obj_rand_swaps, presupuesto_usado_rand_swaps, p)      #   ▸ Reportar

# . Almacenar resultados
resultados.append({
    "Método"             : "Random + Swaps",
    "Empleos generados"  : obj_rand_swaps,
    "Obras seleccionadas": len(obras_selec_rand_swaps),
    "Presupuesto usado"  : presupuesto_usado_rand_swaps,
    "Tiempo (s)"         : end - start
})

Objetos seleccionados:
  • Obj  1: peso=7.0, valor=12.0
  • Obj 23: peso=1.0, valor=12.0
  • Obj 12: peso=8.0, valor=15.0
  • Obj 10: peso=1.0, valor=11.0
  • Obj  0: peso=8.0, valor=13.0
FO (valor total):    63.0
Presupuesto usado:   25.0/25


## Resultados

In [18]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

import pandas as pd

# -- Crear DataFrame desde la lista de resultados --
df_resultados = pd.DataFrame(resultados)

# -- Establecer 'Método' como índice --
df_resultados.set_index("Método", inplace=True)

# -- Ordenar por empleos generados (de mayor a menor) --
df_resultados.sort_values("Empleos generados", ascending=False, inplace=True)

# -- Visualizar resultados ordenados --
df_resultados

,Empleos generados,Obras seleccionadas,Presupuesto usado,Tiempo (s)
Método,,,,
LP,79.0,7,25.0,0.046517
Policy iteration,79.0,7,25.0,0.039916
Value iteration,79.0,7,25.0,0.013112
Greedy (e/c),72.0,7,21.0,0.000044
Random + Swaps,63.0,5,25.0,0.013103
Greedy (e),53.0,4,25.0,0.000051
Random,53.0,5,25.0,0.000068
Policy evaluation,52.0,5,25.0,0.011580


In [19]:
# -- Tomar la solución base de referencia (LP) --
base_lp = df_resultados.loc["LP"]

# -- Calcular el GAP porcentual relativo al método LP (excepto para Tiempo) --
gap_df = ((df_resultados - base_lp) / base_lp) * 100

# -- Reemplazar la columna de tiempo con "x veces LP" en lugar de porcentaje --
tiempo_ratio = (df_resultados["Tiempo (s)"] / base_lp["Tiempo (s)"]).round(4)

# -- Redondear GAPs a 3 decimales --
gap_df = gap_df.round(3)

# -- Renombrar columnas GAP --
gap_df.columns = [col + " GAP (%)" for col in gap_df.columns]

# -- Reemplazar columna de tiempo con "x veces LP" --
gap_df["Tiempo (s) (x veces LP)"] = tiempo_ratio

# -- Eliminar columna anterior de GAP de tiempo --
gap_df = gap_df.drop(columns=["Tiempo (s) GAP (%)"])

# -- Mostrar tabla --
gap_df

,Empleos generados GAP (%),Obras seleccionadas GAP (%),Presupuesto usado GAP (%),Tiempo (s) (x veces LP)
Método,,,,
LP,0.000,0.000,0.0,1.0000
Policy iteration,0.000,0.000,0.0,0.8581
Value iteration,0.000,0.000,0.0,0.2819
Greedy (e/c),-8.861,0.000,-16.0,0.0009
Random + Swaps,-20.253,-28.571,0.0,0.2817
Greedy (e),-32.911,-42.857,0.0,0.0011
Random,-32.911,-28.571,0.0,0.0015
Policy evaluation,-34.177,-28.571,0.0,0.2489


#### Aux

In [20]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_barras_con_etiquetas(df):
    """
    Genera un gráfico de barras con etiquetas de texto dentro de cada barra
    usando Plotly, mostrando tres métricas: empleos, obras y presupuesto.
    """

    # Definir métricas, títulos y colores
    metricas = [
        "Empleos generados",
        "Obras seleccionadas",
        "Presupuesto usado"
    ]
    titulos = [
        "Empleos generados (miles)",
        "Obras seleccionadas",
        "Presupuesto usado (M)"
    ]
    colores = ["#4caf50", "#2196f3", "#ff9800"]  # verde, azul, naranja
    metodos = df.index

    # Crear subplots (1 fila, 3 columnas)
    fig = make_subplots(rows=1, cols=3,
                        subplot_titles=titulos,
                        horizontal_spacing=0.1)

    # Generar cada gráfico
    for i, (col_name, color) in enumerate(zip(metricas, colores), start=1):
        valores = df[col_name]

        # Barras con etiquetas dentro
        fig.add_trace(
            go.Bar(
                x=metodos,
                y=valores,
                text=metodos,
                textposition="inside",   # texto dentro de la barra
                textangle=-90,           # vertical
                insidetextanchor="middle",
                marker_color=color,
                name=col_name,
                textfont=dict(color="white", size=12, family="Arial", weight="bold")
            ),
            row=1, col=i
        )

        # Etiqueta del eje Y
        fig.update_yaxes(title_text=col_name, row=1, col=i)

    # Layout general
    fig.update_layout(
        showlegend=False,
        plot_bgcolor="white",
        paper_bgcolor="white"
    )

    fig.show()

In [21]:
import plotly.express as px

def plot_eficiencia(df):
    """
    Genera un gráfico de dispersión con Plotly:
    Eje X = Tiempo de ejecución (escala log)
    Eje Y = Empleos generados

    Cada punto representa un método distinto.
    
    Parámetros
    ----------
    df : pandas.DataFrame
        Debe contener columnas:
        - 'Tiempo (s)'
        - 'Empleos generados'
        El índice debe contener los nombres de los métodos.
    """

    # Resetear índice y renombrar columna de métodos
    df_plot = df.reset_index().rename(columns={"index": "Método"})

    # Crear gráfico de dispersión
    fig = px.scatter(
        df_plot,
        x="Tiempo (s)",
        y="Empleos generados",
        color="Método",
        size_max=15
    )

    # Ajustes de estilo de puntos
    fig.update_traces(
        marker=dict(size=12, line=dict(width=1, color="black"))
    )

    # Configurar ejes con grid
    fig.update_xaxes(
        type="log",
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False
    )
    fig.update_yaxes(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False
    )

    # Layout general
    fig.update_layout(
        title="Eficiencia: Empleos vs Tiempo",
        xaxis_title="Tiempo de ejecución (s) [escala log]",
        yaxis_title="Empleos generados (miles)",
        plot_bgcolor="white",
        paper_bgcolor="white",
        legend_title="Método"
    )

    fig.show()


In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_gaps(gap_df):
    """
    Gráfico de barras de GAP (%) para empleos, obras y presupuesto usando Plotly.

    Parámetros
    ----------
    gap_df : pandas.DataFrame
        Debe contener columnas:
        - 'Empleos generados GAP (%)'
        - 'Obras seleccionadas GAP (%)'
        - 'Presupuesto usado GAP (%)'
        El índice debe contener los nombres de los métodos.
    """

    # Definición de métricas, títulos y colores
    metricas = [
        "Empleos generados GAP (%)",
        "Obras seleccionadas GAP (%)",
        "Presupuesto usado GAP (%)"
    ]
    titulos = [
        "GAP en empleos (%)",
        "GAP en nº de obras (%)",
        "GAP en presupuesto (%)"
    ]
    colores = ["#4caf50", "#2196f3", "#ff9800"]
    metodos = gap_df.index

    # Crear subplots (1 fila, 3 columnas)
    fig = make_subplots(rows=1, cols=3, subplot_titles=titulos)

    # Crear cada gráfico
    for i, (col_name, color) in enumerate(zip(metricas, colores), start=1):
        valores = gap_df[col_name]

        # Barras con etiquetas
        fig.add_trace(
            go.Bar(
                x=metodos,
                y=valores,
                marker_color=color,
                text=[f"{v:.1f}%" for v in valores],
                textposition="outside",
                showlegend=False
            ),
            row=1, col=i
        )

        # Línea horizontal en y=0
        fig.add_hline(y=0, line_color="black", line_width=1, row=1, col=i)

        # Ejes
        fig.update_yaxes(title_text="Porcentaje", row=1, col=i)

    # Layout general
    fig.update_layout(
        plot_bgcolor="white",
        paper_bgcolor="white",
        title_text="Comparación de GAPs por Método",
        title_x=0.5
    )

    fig.show()


In [23]:
import plotly.graph_objects as go

def plot_tiempo_relativo(gap_df):
    """
    Gráfico de barras horizontales de tiempo relativo respecto a LP usando Plotly.
    - LP se muestra en gris.
    - El resto de métodos en azul.
    - Etiquetas de valores a la derecha de cada barra.
    """

    # Ordenar (aquí se conserva el orden original, puedes ordenar si quieres)
    gap_sorted = gap_df

    # Colores: gris para LP, azul para el resto
    colores = ["#888888" if metodo == "LP" else "#2196f3" for metodo in gap_sorted.index]

    # Límite máximo con margen
    max_val = gap_sorted["Tiempo (s) (x veces LP)"].max()
    limite_x = max_val * 1.2

    # Crear figura
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            x=gap_sorted["Tiempo (s) (x veces LP)"],
            y=gap_sorted.index,
            orientation="h",
            marker_color=colores,
            text=[f"{v:.3f}" for v in gap_sorted["Tiempo (s) (x veces LP)"]],
            textposition="outside"
        )
    )

    # Ajustes de layout
    fig.update_layout(
        xaxis=dict(title="Tiempo relativo (× LP)", range=[0, limite_x]),
        title="Velocidad relativa a LP",
        plot_bgcolor="white",
        paper_bgcolor="white",
        showlegend=False
    )

    fig.show()


#### Comparar

In [24]:
plot_barras_con_etiquetas(df_resultados)
plot_eficiencia(df_resultados)

In [25]:
plot_gaps(gap_df)
plot_tiempo_relativo(gap_df)


## (Opcional) Visualizar PD

In [26]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# Visualización  de V(s)
if env.capacity <= 20:

    # Evaluación de política
    print("Policy Evaluation")
    value_states_visual(env, V, propose_policy)

    # Política óptima vía iteración de políticas
    print("Policy Iteration")
    value_states_visual(env, V_star, star_policy)

    # Política óptima vía iteración de valores
    print("Value Iteration")
    value_states_visual(env, V_opt, opt_policy)

else:
    print("Para visualizar, use una instancia más pequeña.")

Para visualizar, use una instancia más pequeña.


In [27]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# Visualización de las decisiones para cada política evaluada
if env.capacity <= 20:

    # Evaluación de política
    print("Policy Evaluation")
    draw_policy_dag(env, propose_policy, initial_state=(0, env.capacity))
    print()

    # Política óptima vía iteración de políticas
    print("Policy Iteration")
    draw_policy_dag(env, star_policy, initial_state=(0, env.capacity))
    print()

    # Política óptima vía iteración de valores
    print("Value Iteration")
    draw_policy_dag(env, opt_policy, initial_state=(0, env.capacity))
    print()

else:
    print("Para visualizar, use una instancia más pequeña.")


Para visualizar, use una instancia más pequeña.
